# Project 2: Web Traffic Analysis
**This is the second of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-11-10, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
In this project your task is to analyze a stream of log entries. A log entry consists of an [IP address](https://en.wikipedia.org/wiki/IP_address) and a [domain name](https://en.wikipedia.org/wiki/Domain_name). For example, a log line may look as follows:

`192.168.0.1 somedomain.dk`

One log line is the result of the event that the domain name was visited by someone having the corresponding IP address. Your task is to analyze the traffic on a number of domains. Counting the number of unique IPs seen on a domain doesn't correspond to the exact number of unique visitors, but it is a good estimate.

Specifically, you should answer the following questions from the stream of log entries.

- How many unique IPs are there in the stream?
- How many unique IPs are there for each domain?
- How many times was IP X seen on domain Y? (for some X and Y provided at run time)

**The answers to these questions can be approximate!**

You should also try to answer one or more of the following, more advanced, questions. The answers to these should also be approximate.

- How many unique IPs are there for the domains $d_1, d_2, \ldots$?
- How many times was IP X seen on domains $d_1, d_2, \ldots$?
- What are the X most frequent IPs in the stream?

You should use algorithms and data structures that you've learned about in the lectures, and you should provide your own implementations of these.

Furthermore, you are expected to:

- Document the accuracy of your answers when using algorithms that give approximate answers
- Argue why you are using certain parameters for your data structures

This notebook is in three parts. In the first part you are given an example of how to read from the stream (which for the purpose of this project is a remote file). In the second part you should implement the algorithms and data structures that you intend to use, and in the last part you should use these for analyzing the stream.

## Reading the stream
The following code reads a remote file line by line. It is wrapped in a generator to make it easier to extend. You may modify this if you want to, but your solution should remain parametrized, so that your notebook can be run without having to consume the entire file.

In [0]:
import urllib

def stream(n):
    i = 0
    with urllib.request.urlopen('https://files.dtu.dk/fss/public/link/public/stream/read/traffic_2?linkToken=_DcyO-U3MjjuNzI-&itemName=traffic_2') as f:
        for line in f:
            element = line.rstrip().decode("utf-8")
            yield element
            i += 1
            if i == n:
                break

### Hash Function:

In [2]:
!pip install mmh3
import mmh3

### Ends Here

In [0]:
import re

# QUESTION ONE




## How many unique IPs are there in the stream?

Now first we define the function for estimating the cardinality. to do that we use the HyperLogLog data sketch method. The main idea of the and the parameters were drawn from the orinial paper of the HyperLogLog :

HyperLogLog: the analysis of a near-optimal cardinality estimation algorithm
Philippe Flajolet, Éric Fusy, Olivier Gandouet, Frédéric Meunier


The following open sources as well was used to make the alghorithem:


cardinality estimation algorithm
Philippe Flajolet, Éric Fusy, Olivier Gandouet, Frédéric Meunier

The following open sources as well was used to make the alghorithem:

hyperloglog, HLL (both available in pipy.org)

In [0]:
import re
import struct, copy
import numpy as np
import warnings
import hashlib


def sha1_hash32(data):
   
    return struct.unpack('<I', hashlib.sha1(data).digest()[:4])[0]


_bit_length = lambda bits : bits.bit_length()


class HyperLogLog_webt(object):
    
    __slots__ = ('p', 'm', 'reg', 'alpha', 'max_rank', 'hashfunc')
    
    _hash_range_bit = 32
    _hash_range_byte = 4

    def _get_alpha(self, p):
        return 0.7213 / (1.0 + 1.079 / (1 << p))

    def __init__(self, p=8, reg=None, hashfunc=sha1_hash32, hashobj=None):
        if reg is None:
            self.p = p
            self.m = 1 << p
            self.reg = np.zeros((self.m,), dtype=np.int8)
        self.hashfunc = hashfunc
        self.alpha = self._get_alpha(self.p)
        self.max_rank = self._hash_range_bit - self.p

    def update(self, b):
        
        hv = self.hashfunc(b)
        reg_index = hv & (self.m - 1)
        bits = hv >> self.p
        self.reg[reg_index] = max(self.reg[reg_index], self._get_rank(bits))

    def count(self):
        
        e = self.alpha * float(self.m ** 2) / np.sum(2.0**(-self.reg))
        small_range_threshold = (5.0 / 2.0) * self.m
        
        if e <= small_range_threshold:
            num_zero = self.m - np.count_nonzero(self.reg)
            return self._linearcounting(num_zero)
        if e <= (1.0 / 30.0) * (1 << 32):
            return e
        return self._largerange_correction(e)
    def _get_rank(self, bits):
        rank = self.max_rank - _bit_length(bits) + 1
        return rank
    def _linearcounting(self, num_zero):
        return self.m * np.log(self.m / float(num_zero))

In [5]:
1 << 8

256

Let's have a small test on the function, we build a list containing the 6 unique names adding two repeatative names and compare the the function response with the real number of unique component of the list.

In [6]:

Prof_TA = ['Philip Bille', 'Patrick Hagge Cording', 'Inge Li Gørtz', 'Inge Li Gørtz',
'Bastian Ellegård Grønager', 'Max Harry Rishøj Pedersen', 'Sarah Alexandra Maria Van Dam','Sarah Alexandra Maria Van Dam' ]

h = HyperLogLog_webt()
for d in Prof_TA:
  h.update(d.encode('utf8'))
print("Estimated cardinality is", h.count())
s1 = set(Prof_TA)
print("Actual cardinality is", len(s1))

Estimated cardinality is 6.0714308140329125
Actual cardinality is 6


Now, with the Hyperloglog_webt we could analyze the cardinality of the web stream:

In [7]:
STREAM_SIZE = 10000
web_traffic_stream = stream(STREAM_SIZE)
##
h = HyperLogLog_webt()
listsave =[]
for l in web_traffic_stream:
    ip = re.findall( r'[0-9]+(?:\.[0-9]+){3}', l)[0]
    listsave.append(ip)
    h.update(ip.encode('utf8'))
print("Estimated cardinality is", h.count())
s1 = set(listsave)
print("Actual cardinality is", len(s1))

Estimated cardinality is 9482.081530493413
Actual cardinality is 9985


It must be noted that the error of the algorithm is typical (based on paper):


Expected_Value +- (1.04/sqrt(m))

In [8]:
relative_error = 1.04/(1 << 8)**0.5
relative_error

0.065

# QUESTION TWO

##How many unique IPs are there for each domain?

For answer this question, we will use the FM Sketch algoritm https://www.cs.helsinki.fi/u/jilu/paper/FMSketch.pdf 
This will help us to approximate the number of distinct elements in a
stream with a single pass and space efficient.

We will define the size of the domain of our hash function HASH_DOMAIN_SIZE, that is the total different hashes values that we can get from applying the hash function

For purposes of providing examples and quick overviews, we will use in some cases fix size of streaming. 

In [0]:
def string_hash(word,n, seed):
  return sum(ord(char) for char in word)%n
  return mmh3.hash(word,seed)%n

In [0]:
def trailing_zeros(longint):
    manipulandum = str(longint)
    return len(manipulandum)-len(manipulandum.rstrip('0'))

In [0]:
#Function that returns the position of the first zero in an array
def first_zero(array):
    for i, value in enumerate(array):
        if value==0:
            return i
    return len(array)+1

In [0]:
HASH_DOMAIN_SIZE=100


def FM_sketch(stream):
    #We iterate for every URL
    #We create a dictionary for saving every URL we find
    dict_webs = {}  
    for l in stream:
        #We get the ip form the string
        ip=re.findall( r'[0-9]+(?:\.[0-9]+){3}', l )
        if(len(ip)>=1):
            #First we check if the current URL is in the dictonary
            url=l.split()[1]
            if url not in dict_webs:
                #We initialize the FM array
                FM_array = [0] * HASH_DOMAIN_SIZE
                dict_webs[url]=FM_array
            #We retrieve the array from the dictionary
            FM_array=dict_webs[url]
            #We transform the hashed ip to binary, and then we get the tail of the binary value
            FM_array[trailing_zeros(bin(string_hash(ip[0],HASH_DOMAIN_SIZE, 41)))]=1
            #We save the updated FM_array in the dictionary
            dict_webs[url]=FM_array
    dict_webs_count={}
    #We last compute the estimated value
    for url in dict_webs.keys():
        FM_array=dict_webs[url]
        #Where 0.77351 is the correlation factor
        dict_webs_count[url]=2**first_zero(FM_array)/ 0.77351
    return dict_webs_count

### This give us a dictionary with the different URL and the estimated number of unique IPs. This is an aproximate solution, because we are based on statistical estimation approximating computation with a smaller space with logarithmic complexity in the number of distinct elements.

In [13]:
my_stream=stream(1000)


FM_sketch(my_stream)

{'databricks.com': 20.6849297358793,
 'datarobot.com': 2.5856162169849126,
 'dtu.dk': 82.7397189435172,
 'github.com': 41.3698594717586,
 'google.com': 20.6849297358793,
 'pandas.pydata.org': 82.7397189435172,
 'python.org': 165.4794378870344,
 'spark.apache.org': 5.171232433969825,
 'wikipedia.org': 165.4794378870344}

# QUESTION THREE

##How many times was IP X seen on domain Y? (for some X and Y provided at run time)

### Here we seek the number of times a given IP address appears on a particular domain. 
### A number of hash functions are created. Subsequently, a dictionary is also created in order to generate keys that will be hashed. 

### We increase the value of every hash in its corresponding matrix. First we check if the current URL is in the dictonary, if it is, we will use the current matrix, if not, we create a new one and so on.



In [0]:
import pandas as pd
import mmh3
import numpy as np
import re

In [0]:
STREAM_SIZE = 10000
customer_stream = stream(STREAM_SIZE) 

In [0]:
def hash1(string):
  return string_hash(string,100, 1)
def hash2(string):
  return string_hash(string, 100, 12) 
def hash3(string):
  return string_hash(string, 100, 23) 
def hash4(string):
  return string_hash(string, 100, 34) 
def hash5(string):
  return string_hash(string,100, 40)
def hash6(string):
  return string_hash(string, 100, 50) 
def hash7(string):
  return string_hash(string, 100, 60) 
def hash8(string):
  return string_hash(string, 100, 77) 

In [0]:
import numpy as np
import re

HASH_DOMAIN_SIZE=1000 #max number of rows

def countMin_sketch(hashes, stream):
  #We create the matrix and set its values to 0
  
  #We create a dictionary for saving every URL we find
  dict_webs = {}  
  for l in stream:
    for indx,hash in enumerate(hashes):
      #We get the ip form the string
      ip=re.findall( r'[0-9]+(?:\.[0-9]+){3}', l )
      if(len(ip)>=1):
        #We increase the value of every hash in its corresponding matrix
        #First we check if the current URL is in the dictonary, if it is, we will use the current matrix, if not, we create a new one
        url=l.split()[1]
        if url in dict_webs:
            #We update the Matrix
            M=dict_webs[url]
            M[indx][hash(ip[0])]=M[indx][hash(ip[0])]+1
            dict_webs[url]=M
        else:
            #We create a new Matrix
            M = np.zeros((len(hashes), HASH_DOMAIN_SIZE))
            M[indx][hash(ip[0])]=M[indx][hash(ip[0])]+1
            dict_webs[url]=M
  return dict_webs

In [0]:
hashes = [hash1, hash2, hash3, hash4,hash5, hash6, hash7, hash8]
my_stream=stream(500)

M=countMin_sketch(hashes, my_stream)

###Let's consider an instance to test our function. We have manually inserted a domain, that is domain Y and IP X to test our function. The result is impressive  for a given fixed data, the number of times the given IP address visited a given domain is very accurate in this test results. Through this, our function can process a data stream. 

### Equally, important, it is found that when the number of hashes increases, the accuracy of the function improves greatly. However, it must be stressed that there is a trade-off between accuracy and space usage. This means, we are sacrificing space for accuracy. 

In [19]:
def count_ip(url, ip):
  array=[]
  for index,hash in enumerate(hashes):
    array.append(M[url][index][hash(ip)])
  return min(array)

count_ip('python.org', "203.85.185.49")

#https://florian.github.io/count-min-sketch/


4.0